In [1302]:
#import libraries
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchFrameException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.action_chains import ActionChains

# import configparser
import mysql.connector

print('Libraries Imported.')

Libraries Imported.


In [1303]:
# retruns a new web driver object
def createNewDriver():   

    # run with our without browser window
    options = webdriver.ChromeOptions() 
    # options.add_argument('--headless') # run driver without browser
    driver = webdriver.Chrome(executable_path='C:\Program Files\chromedriver\chromedriver.exe', 
                                options=options)

    return driver

In [1304]:
# navigate driver to proper frame within Product Line tab
def navigateToProductList(driver):
    url = 'https://www.echo-usa.com/Support-Help/Parts-Lookup'

    driver.get(url)

    # navigate to product line tab
    frameset = driver.find_element_by_tag_name('frameset')

    frame = frameset.find_element_by_tag_name('frame')
    frame_name = frame.get_attribute('name')

    # grab last 6 chars of frame name since it changes everytime page is loaded
    frame_suffix = frame_name[11:17]
    # print(frame_suffix)

    # set driver in frame that contains necessary info
    driver.switch_to.frame(frame)

    # find product line tab
    product_line = driver.find_element_by_id("Tab2")

    # select product line tab
    product_line.click()
    # print(browser.page_source)

    # set names/IDs of frames based on 6 chars that were previously saved
    prodLineFrameID = 'iframeSearchProdLine_' + frame_suffix
    detailFrameID = 'plDetailFrame'
    dataFrameName = 'ARICAT6TREE_' + frame_suffix

    # navigate into frame that contains ProductLine
    frame = driver.find_element_by_id(prodLineFrameID)
    driver.switch_to.frame(frame)

    frame = driver.find_element_by_id(detailFrameID)
    driver.switch_to.frame(frame)

    frame = driver.find_element_by_name(dataFrameName)
    driver.switch_to.frame(frame)

    # click into ECHO jstree
    cur_node = driver.find_element_by_id('pn18')
    cur_node.click()

    return frame_suffix

In [1305]:
MANUFACTURER = 'Echo'

# define id variables outside of functions so that they can be accessed without being passed in
tool_id = 0
attachment_id = 0
tool_diagram_id = 0
attachment_diagram_id = 0
tool_component_id = 0

# iterates through nodes within Product List,
# unorderedList: a web element referencing a UL
# is_attachment: a bool indicating if the function is currently within an attachment file
def getNodes(unorderedList, is_attachment):

    # is_attachment_next is used for subsequent lower levels, so that is_attachment remains accurate for current level
    # is_attachment is for current level
    is_attachment_next = is_attachment

    # nodes of unorderedList
    nodes = unorderedList.find_elements_by_tag_name("li")

    # iterate through nodes
    for node in nodes:      
        # node ID
        node_id = node.get_attribute('id')

        # Skip Attachments and Accessories
        if node_id == "assemTree-item-.18.19":
            continue
        
        # wait for element to be present, get fresh reference
        wait = WebDriverWait(driver, 10)
        node = wait.until(EC.presence_of_element_located((By.ID, node_id)))

        # ID of node anchor
        anchor_id = node_id + '_anchor'

        # wait for anchor to be visible and clickable
        wait = WebDriverWait(driver, 10)
        wait.until(EC.visibility_of_element_located((By.ID, anchor_id)))
        anchor = wait.until(EC.element_to_be_clickable((By.ID, anchor_id)))

        # click anchor to expand nested list
        anchor.click()
        
        # store anchor text
        text = anchor.text

        # wait for node to be present again, things may have changed after clicking
        ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)    
        wait = WebDriverWait(driver, 10, ignored_exceptions=ignored_exceptions)
        node = wait.until(EC.presence_of_element_located((By.ID, node_id)))

        # call process to add data to SQL if necessary 
        processNode(node, node_id, text, is_attachment, is_attachment_next)

In [1314]:
# processNode discovers what type of node 'node' is and adds any necessary data to SQL or passes necessary data to other functions to be handled
# node: web element, reference to current node
# node_id: string, id of current node
# is_attachment: bool, whether or not current node is within an attachment file
# is_attachment_next: whether subsequent nodes are also in an attachment file
def processNode(node, node_id, text, is_attachment, is_attachment_next):
    global tool_id
    global attachment_id
    global tool_diagram_id
    global attachment_diagram_id

    # create cursor to execute SQL statements
    myCursor = tool_db.cursor(buffered=True)

    # use a try/except to catch StaleElementException, then 
    # grab a new reference to the element that casued the 
    # exception and try again
    try:
        # check if node is an Attachments subfile
        if node.get_attribute('data-assembly-type') =="AT":
            is_attachment_next = True
        # check if node is a tool model or attachment model
        elif node.get_attribute('data-assembly-type') == "MD":
            # find substring 'S/N' for serial number range
            serialNumIndex = text.find('S/N')
            # if 'S/N' is found
            if (serialNumIndex != -1):
                # serial number range starts at serialNumIndex + 5
                sn_range = text[serialNumIndex + 5:]
                # everything before serialNumIndex is the model name
                model = text[0 : serialNumIndex -1]
            # if 'S/N' not in string, full string is model name
            else:
                model = text
                sn_range = None
            
            # add tool into tools table
            if is_attachment is False:
                tool_id += 1
                sql = ('INSERT INTO tools (tool_id, manufacturer, model, serial_number_range) VALUES (%s, %s, %s, %s)')
                vals = (tool_id, MANUFACTURER, model, sn_range)
            # add attachment to attachments table
            else:
                attachment_id += 1
                sql = ('INSERT INTO attachments (attachment_id, tool_id, manufacturer, model) VALUES (%s, %s, %s, %s)')
                vals = (attachment_id, tool_id, MANUFACTURER, model)
            # execute and commit SQL statements
            myCursor.execute(sql, vals)
            tool_db.commit()

        # base case check if node is a parent, if false gather info from leaf node
        # need to check thumb-url in case there is a false leaf (item is leaf node but contains no components)
        elif node.get_attribute('data-assembly-type') == "IP" and node.get_attribute('data-assembly-thumb-url') is not None:
            # No Diagrams Required means there are no components or further info on the diagram
            if text == 'No Diagrams Required':
                return
            # otherwise insert the diagram into tool_diagrams or attachment_diagrams table
            else:
                # add diagram to tool_diagrams table
                if is_attachment is False:
                    tool_diagram_id += 1
                    sql = ('INSERT INTO tool_diagrams (diagram_id, tool_id, manufacturer, description) VALUES (%s, %s, %s, %s)')
                    vals = (tool_diagram_id, tool_id, MANUFACTURER, text)
                # add diagram to attachment_diagrams table
                else:
                    attachment_diagram_id += 1
                    sql = ('INSERT INTO attachment_diagrams (diagram_id, attachment_id, manufacturer, description) VALUES (%s, %s, %s, %s)')
                    vals = (attachment_diagram_id, attachment_id, MANUFACTURER, text)
                # execute and commit SQL statements
                myCursor.execute(sql, vals)
                tool_db.commit()
                
                # since node was a diagram, check for components and add them to related component table
                getComponents(is_attachment, tool_diagram_id, attachment_diagram_id, myCursor)

                # node path is complete, return to process next node
                return
        # get xpath of current node
        xpath = '//li[@id = \'' + node_id + '\' and @aria-expanded = \'true\' and @aria-busy = \'false\']'

        # get new reference to node in case DOM changed
        wait = WebDriverWait(driver, 20)
        node = wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))

        # find next unordered list element and call getParts recursively to iterate through next level
        getNodes(node.find_element_by_tag_name("ul"), is_attachment_next)

    # catch StaleElementReferenceException and get a new reference to the element and retry
    except StaleElementReferenceException:
        # print("STALE :(")
        # print("Tool ID: ", tool_id, " Tool_Diagram_ID: ", tool_diagram_id)
        # print("Attachment_ID: ", attachment_id, " Attachment_Diagram_ID: ", attachment_diagram_id)
        # print()
        
        ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)    
        wait = WebDriverWait(driver, 10, ignored_exceptions=ignored_exceptions)
        node = wait.until(EC.presence_of_element_located((By.ID, node_id)))

        processNode(node, node_id, text, is_attachment, is_attachment_next)

    except ElementNotInteractableException as err:
        print(err)
        print(tool_id)
        print(attachment_id)
        print(attachment_diagram_id)
        print(tool_diagram_id)

        # ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)    
        # wait = WebDriverWait(driver, 10, ignored_exceptions=ignored_exceptions)
        # node = wait.until(EC.presence_of_element_located((By.ID, node_id)))

        # processNode(node, node_id, text, is_attachment, is_attachment_next)
        return

    # finally:
    #     myCursor.close()
    #     tool_db.close()
        

In [1315]:
# getComponents navigates to the component list frame if it exists and adds components to respective tables in SQL
# is_attachment: bool, if current node is in attachment file
# tool_diagram_id: int, diagram id for SQL
# attachment_diagram_id: attachment diagram id for SQL
# myCursor: cursor object to execute SQL
def getComponents(is_attachment, tool_diagram_id, attachment_diagram_id, myCursor):
    # switch to outer frame
    driver.switch_to.parent_frame()
    
    frame = driver.find_element_by_name('plDetail')
    
    # navigate to frame that contains components
    driver.switch_to.frame(frame)
    driver.switch_to.frame(2)

    # find table
    table = driver.find_element_by_tag_name('table')

    manufacturer = 'Echo'

    sku = None
    description = None
    
    # iterate table
    for row in table.find_elements_by_css_selector('tr'):
        # use index to reference specific cells in table
        index = 0
        for cell in row.find_elements_by_css_selector('td'):
            # get sku
            if index == 3:
                sku = cell.text
            # get description
            elif index == 4:
                description = cell.text
            index += 1
        # func was adding two null entries at the start of every new list of components
        # this seems to negate that
        if sku == None and description == None:
            continue

        # TO DO: create function for repeated code below

        # add components to tool_component tables and tool_diagram_component_map
        if is_attachment is False:
            # add new component to table if it isn't already
            if not checkComponentExists(sku, 'tool_components', myCursor):
                sql = ('INSERT INTO tool_components (manufacturer, sku, description) VALUES (%s, %s, %s)')
                vals = (manufacturer, sku, description)
                myCursor.execute(sql, vals)
                tool_db.commit()

            # get tool_component_id from SQL
            tool_component_id = getComponentID(sku, 'tool_components', myCursor)

            # check if map relationship already exists, some components are repeated in the same diagram
            if not checkMapEntryExists(int(tool_diagram_id), int(tool_component_id), 'tool_diagram_component_map', myCursor):

                sql = ('INSERT INTO tool_diagram_component_map (diagram_id, component_id) VALUES (%s, %s)')
                vals = (tool_diagram_id, tool_component_id)

                myCursor.execute(sql, vals)
                tool_db.commit()
        # add components to attachment_components and attachment_diagram_component_map
        else:
            # add new component to table if it isn't already
            if not checkComponentExists(sku, 'attachment_components', myCursor):
                sql = ('INSERT INTO attachment_components (manufacturer, sku, description) VALUES (%s, %s, %s)')
                vals = (manufacturer, sku, description)
                myCursor.execute(sql, vals)
                tool_db.commit()
            # get attachment_component_id
            attachment_component_id = getComponentID(sku, 'attachment_components', myCursor)

            # check if map relationship already exist, some components are repeated in the same diagram
            if not checkMapEntryExists(int(attachment_diagram_id), int(attachment_component_id), 'attachment_diagram_component_map', myCursor):
                sql = ('INSERT INTO attachment_diagram_component_map (diagram_id, component_id) VALUES (%s, %s)')
                vals = (attachment_diagram_id, attachment_component_id)
                
                myCursor.execute(sql, vals)
                tool_db.commit()

    # after getting all components, return to product list to continue iteration
    returnToProductList()


In [1308]:
# checks if relationship already exists between diagram and component
# diagram_id: int
# component_id: int
# table: string, table name to check
# myCursor: cursor to execute SQL statements
def checkMapEntryExists(diagram_id, component_id, table, myCursor):
    sql = ("SELECT * FROM `" + table + "` WHERE diagram_id = %s AND component_id = %s")
    vals = (diagram_id, component_id)

    myCursor.execute(sql, vals)
    # returns results of query
    return myCursor.fetchall()

In [1309]:
# checks if component exists in respective component table
# sku: string, sku from component
# table: string, name of table to query
# myCursor: cursor object
def checkComponentExists(sku, table, myCursor):
    # check sku
    if sku != '--' and sku != None:
        sql = ("SELECT * FROM `" + table + "` WHERE EXISTS(SELECT * FROM `" + table + "` WHERE sku = '" + sku + "')")

        myCursor.execute(sql)

        # return results
        return myCursor.fetchall()
    # return false since it can't be verified the component already exists
    else:
        return False

In [1317]:
# gets the id of the component after it is enterd into table
# sku: string
# table: string: table name to be queried
# myCursor: cursor to execute SQL statements
def getComponentID(sku, table, myCursor):
    # if sku is -- then it is assumed to be unique and we can grab the newest entry 
    # in the table since it should have been just entered
    if sku == "--":
        sql = ("SELECT component_id FROM `" + table + "` ORDER BY component_id DESC LIMIT 1")
        myCursor.execute(sql)
        return myCursor.fetchone()[0]

    # otherwise search table for the sku and return the id
    sql = ("SELECT component_id FROM `" + table + "` WHERE sku = '" + sku + "'")
    myCursor.execute(sql)
    return myCursor.fetchone()[0]

In [1316]:
# returns program to product list frame to continue iteration
def returnToProductList():
    driver.switch_to.parent_frame()
    driver.switch_to.parent_frame()

    frameName = 'ARICAT6TREE_' + frame_suffix

    # navigate into frame that contains product list
    frame = driver.find_element_by_name(frameName)
    driver.switch_to.frame(frame)


In [1312]:
# connect to tool_scraping database
def connectMySQL():
    HOST = "ifixit-hiring.cxbmajqbo6pd.us-west-1.rds.amazonaws.com"
    USERNAME = "admin"
    PASSWORD = "mq76CKehrqTY7hgQZDRU"
    DATABASE = "tool_scraping"

    tool_db = mysql.connector.connect(
        host = HOST,
        user = USERNAME,
        password = PASSWORD,
        database = DATABASE
    )

    return tool_db

In [1313]:
# create new webdriver to begin exploring target data
driver = createNewDriver()
frame_suffix = navigateToProductList(driver)

tool_db = connectMySQL()

# select first ul that contains all subdirectories
productList = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//ul[contains(@class, \'jstree-container-ul\')][li[@aria-level = \'1\']]')))

# pass list of products into getNodes(), a recursive func
getNodes(productList, False)

0 Size :(
24
102
92
305
Message: element not interactable: element has zero size
  (Session info: chrome=95.0.4638.54)

0 Size :(
24
106
95
305
Message: element not interactable: element has zero size
  (Session info: chrome=95.0.4638.54)

0 Size :(
24
106
95
308
Message: element not interactable: element has zero size
  (Session info: chrome=95.0.4638.54)

0 Size :(
25
115
104
308
Message: element not interactable: element has zero size
  (Session info: chrome=95.0.4638.54)

0 Size :(
25
119
107
308
Message: element not interactable: element has zero size
  (Session info: chrome=95.0.4638.54)

0 Size :(
25
119
107
311
Message: element not interactable: element has zero size
  (Session info: chrome=95.0.4638.54)

0 Size :(
26
130
117
311
Message: element not interactable: element has zero size
  (Session info: chrome=95.0.4638.54)

0 Size :(
26
134
120
311
Message: element not interactable: element has zero size
  (Session info: chrome=95.0.4638.54)

0 Size :(
26
134
120
314
Message: e

NoSuchFrameException: Message: no such frame
  (Session info: chrome=95.0.4638.54)
